# 1. Read and pre-process the data

## 1.1 Read data

This is the first part of the code and is designed for checking and cleaning the data. The data used is taken from Alberta Water Quality Data Portal with focus on water matrix (0) and long-term and tributary monitoring stations:
Source for getting data: https://environment.extranet.gov.ab.ca/apps/WaterQuality/dataportal/DataDownload/Index/

In [1]:
import numpy as np
import pandas as pd
import os
import sys
# add src folder to path
sys.path.append(os.path.abspath('../src'))

# import funcs from src folder in higher directory
from funcs import unit_harm

In [2]:
# Specify parse date:
parse_dates = ['SampleDateTime']

# Now read the data
data = pd.read_csv('../data/Water Quality-2025-03-08 172848.csv', dtype=
{
 'ProjectNumber': 'category',
 'SampleNumber': 'category',
 'ContinentalRiverBasinCode': 'category',
 'RiverBasinCode': 'category',
 'RiverSubBasinCode': 'category',
 'StationTypeCode': 'category',
 'StationNumber': 'category',
 'Station': 'category',
 'LatitudeDecimalDegrees': 'float64',
 'LongitudeDecimalDegrees': 'float64',
 'SampleMatrixCode': 'category',
 'SampleTypeCode': 'category',
 'CollectionCode': 'category',
 'QCSampleFlag': 'category',
 'SampleComment': 'string',
 'SampleDateTime': 'string',
 'VmvCode': 'category',
 'VariableCode': 'category',
 'VariableName': 'category',
 'MeasurementFlag': 'category',
 'MeasurementValue': 'float64',
 'UnitCode': 'category',
 'SampleDetectLimit': 'string',
 'MeasurementComment': 'string',
 'MeasurementQualifier': 'category',
 'MeasurementQualifierDescription': 'string',
 'MeasurementQualifierComment': 'string',
 'MethodCode': 'category',
 'MethodDetectionLimit': 'float64',
 'LabCode': 'category'
 }
 , na_values=['', 'NaN', 'NULL', 'N/A', 'NA', 'null', '******', 'NaT', 'nan'])

In [3]:
# convert the SampleDateTime column to datetime
data['SampleDateTime'] = pd.to_datetime(data['SampleDateTime'], format='%m/%d/%Y %H:%M:%S', errors='coerce')

# Convert SampleDetectLimit to numeric
data['SampleDetectLimit'] = pd.to_numeric(data['SampleDetectLimit'], errors='coerce')

# convert VariableName and Station to title case
data[['VariableName', 'Station']] = data[['VariableName', 'Station']].apply(lambda x: x.str.title())

# get rid of whitespace
data['VariableName'] = data['VariableName'].str.strip()

# make units lowercase
data['UnitCode'] = data['UnitCode'].str.lower()

# 1.2 Filter the data

In [4]:
# Check missingness for each column and arrange in descending order
data.isnull().sum().sort_values()

ProjectNumber                            0
VmvCode                                  0
SampleDateTime                           0
MeasurementValue                         0
QCSampleFlag                             0
MethodCode                               0
SampleTypeCode                           0
SampleMatrixCode                         0
VariableCode                             0
LongitudeDecimalDegrees                  0
Station                                  0
StationNumber                            0
StationTypeCode                          0
RiverSubBasinCode                        0
RiverBasinCode                           0
ContinentalRiverBasinCode                0
SampleNumber                             0
LatitudeDecimalDegrees                   0
VariableName                             0
LabCode                                288
UnitCode                             73616
CollectionCode                      200683
MethodDetectionLimit                386180
SampleComme

**Note:** it is the user's choice to not include the data without unit codes. I chose to eliminate them as they really create ambiguity in how usable the data will be. Just as a more general rule, we apply SampleDateTime, MeasurementValue, StationNumber, and VariableName. 

In [5]:
data = data.dropna(subset = ['UnitCode', 'MeasurementValue', 'SampleDateTime', 'StationNumber', 'VariableName'])
data = data.dropna(how = 'all') # drop rows where all elements are NaN

In [6]:
duplicates = data.duplicated(subset=['SampleDateTime', 'StationNumber', 'VariableName', 'MeasurementValue', 
                                     'UnitCode', 'VmvCode', 'SampleNumber', 'LabCode'], 
                             keep=False)
                             
# write duplicate rows to csv   
# data[duplicates].to_csv('../output/duplicate_rows.csv', index=False)

Basically there is no duplicate row based on the analysis of duplicates. 

# 1.3 Harmonize the units
Harmonize the units so that each parameter has only one unit. This is helpful for creating plots and comparing data against other stations or time-windows.

In [7]:
data = unit_harm(data)

1,1,1,2-Tetrachloroethane
1,1,1-Trichloroethane
1,1,2,2-Tetrachloroethane
1,1,2-Trichloroethane
1,1-Dichloroethane
1,1-Dichloroethylene
1,1-Dichloropropylene
1,2,3-Trichlorobenzene
1,2,3-Trichloropropane
1,2,4-Trichlorobenzene
1,2,4-Trimethylbenzene
1,2,6-Trimethylphenanthrene
1,2-Dibromo-3-Chloropropane
1,2-Dibromoethane
1,2-Dichlorobenzene
1,2-Dichloroethane
1,2-Dichloroethene-Cis
1,2-Dichloroethene-Trans
1,2-Dichloropropane
1,2-Dimethylnaphthalene
1,2-Diphenylhydrazine
1,2-Xylene
1,3,5-Trichlorobenzene
1,3,5-Trimethylbenzene
1,3-Dichlorobenzene
1,3-Dichloropropane
1,3-Dichloropropene-Cis
1,3-Dichloropropene-Trans
1,4,6,7-Tetramethylnaphthalene
1,4-Dichloro-2-Butene-Cis
1,4-Dichloro-2-Butene-Trans
1,4-Dichlorobenzene
1,4-Dioxane
1,7-Alpha-Ethynylestradiol
1,7-Dimethylfluorene
1,7-Dimethylphenanthrene
1,8-Dimethylphenanthrene
1-Methylchrysene
1-Methylnaphthalene
1-Methylphenanthrene
12,14-Dichlorodehydroabietic Acid
12-Chlorodehydroabietic Acid
14-Chlorodehydroabietic Acid
2,2-Dichlor

In [8]:
# Check those variables with more than one unit
unit_counts = data.groupby('VariableName')['UnitCode'].nunique().reset_index()
pars_with_multiple_units = unit_counts.loc[unit_counts['UnitCode'] > 1, 'VariableName'].tolist()

for i in pars_with_multiple_units:
    print(i)
    print(data[data['VariableName'] == i].groupby('UnitCode')['UnitCode'].count())
    print('-----------------------------------------')

2,3,6-Trichlorophenol (Surrogate)
UnitCode
%       53
ug/l    46
Name: UnitCode, dtype: int64
-----------------------------------------
Coliforms Fecal
UnitCode
cfu/100 ml      134
no/100 ml     21085
Name: UnitCode, dtype: int64
-----------------------------------------
Coliforms Total
UnitCode
mpn/100 ml       3
no/100 ml     3061
Name: UnitCode, dtype: int64
-----------------------------------------
Discharge Daily Mean
UnitCode
cfs     212
m3/s     49
Name: UnitCode, dtype: int64
-----------------------------------------
Discharge Instantaneous
UnitCode
cfs      31
m3/s    219
Name: UnitCode, dtype: int64
-----------------------------------------
Escherichia Coli
UnitCode
cfu/100 ml      405
mpn/100 ml        3
no/100 ml     18098
Name: UnitCode, dtype: int64
-----------------------------------------
Fluorescent Dissolved Organic Matter-Fdom (Field)
UnitCode
ppb qsu    209
rfu        209
Name: UnitCode, dtype: int64
-----------------------------------------
Sampling Distance From L

## 1.4 Handing censored data and adding date stamps

In [9]:
# Add censored column
data['Censored'] = data.apply(
    lambda row: True if row['MeasurementFlag'] == 'L'
    else False if (row['MeasurementFlag'] != 'L' and pd.isna(row['SampleDetectLimit']))
    else row['MeasurementValue'] < row['SampleDetectLimit'], 
    axis=1
)

# Extract year, day, month column from SamepleDateTime:
data['Year'] = data['SampleDateTime'].dt.year
data['Month'] = data['SampleDateTime'].dt.month
data['Day'] = data['SampleDateTime'].dt.day

# Add month names and make them categorical
data['MonthCategory'] = data['SampleDateTime'].dt.month_name()
data['MonthCategory'] = pd.Categorical(data['MonthCategory'],
                                                categories=['January', 'February', 'March', 'April', 'May', 'June',
                                                            'July', 'August', 'September', 'October', 'November', 'December'],
                                                ordered=True)

# Add month short names and make them categorical
data['MonthCategoryShort'] = data['SampleDateTime'].dt.strftime("%b")
data['MonthCategoryShort'] = pd.Categorical(data['MonthCategoryShort'],
                                                categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                                            'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                                                ordered=True)

## 1.5 EDA

In [10]:
data_backup = data

In [18]:
data = data_backup.copy()
data.describe()

,LatitudeDecimalDegrees,LongitudeDecimalDegrees,SampleDateTime,MeasurementValue,SampleDetectLimit,MethodDetectionLimit,Year,Month,Day
count,2.209871e+06,2.209871e+06,2209871,2.209871e+06,1676151.0,1.897307e+06,2.209871e+06,2.209871e+06,2.209871e+06
mean,5.252154e+01,-1.136532e+02,2012-09-21 09:48:36.623111680,3.953181e+02,11.246677,1.337832e+01,2.012236e+03,6.401583e+00,1.478128e+01
min,4.902673e+01,-1.188047e+02,1959-02-20 14:00:00,-2.130000e+02,0.0,0.000000e+00,1.959000e+03,1.000000e+00,1.000000e+00
25%,5.035310e+01,-1.144871e+02,2007-02-20 14:30:00,3.910000e-02,0.005,6.000000e-03,2.007000e+03,4.000000e+00,9.000000e+00
50%,5.208902e+01,-1.134421e+02,2016-02-08 14:15:00,8.410000e-01,0.05,5.000000e-02,2.016000e+03,7.000000e+00,1.500000e+01
75%,5.420120e+01,-1.124759e+02,2020-07-21 10:05:00,1.367000e+01,0.3,8.000000e-01,2.020000e+03,9.000000e+00,2.000000e+01
max,5.844722e+01,-1.100297e+02,2024-12-12 14:45:00,1.330000e+06,80000.0,1.000000e+04,2.024000e+03,1.200000e+01,3.100000e+01
std,2.512963e+00,1.996642e+00,NaN,6.866319e+03,149.601908,1.668303e+02,1.028575e+01,3.088282e+00,7.027856e+00


## 1.6 Join with site IDs for upstream to downstream plottins

In [19]:
siteIDs = pd.read_csv('../data/site_IDs.csv')
data = data.merge(siteIDs, how='left', on='StationNumber')
data = data.sort_values(by=["Basin", "PLOT_CODE2"])

In [ ]:
data['StationNumber'] = pd.Categorical(data['StationNumber'], ordered=True, categories=data['StationNumber'].unique())
data['SITE_ID'] = pd.Categorical(data['SITE_ID'], ordered=True, categories=data['SITE_ID'].unique())
data['PLOT_CODE'] = pd.Categorical(data['PLOT_CODE'], ordered=True, categories=data['PLOT_CODE'].unique())
data['Station'] = pd.Categorical(data['Station'], ordered=True, categories=data['Station'].unique())

# check values to make sure the order is correct:
print(list(data['SITE_ID'].cat.categories))

['SAR', 'MTR', 'AR1', 'AR2', 'BER', 'SAK', 'MCL', 'AR3', 'PBR', 'LSR', 'AR4', 'AR5', 'AR6', 'AR7', 'AR8', 'BTR1', 'BTR2', 'BVR1', 'BVR2', 'BVR3', 'GR', 'WSC', 'JPC', 'BR1', 'ER', 'NC', 'FC', 'PC', 'SR1', 'SR2', 'HR1', 'HR2', 'BR2', 'WAC', 'EAC', 'BR3', 'CFC', 'CC', 'NWC', 'TMC', 'BR4', 'NMR', 'MLK1', 'MLK2', 'RC', 'VC', 'MSC', 'MLK3', 'NSR1', 'SFR', 'CLNR', 'BGHR', 'NSR2', 'RMR', 'NSR3', 'CLWR2', 'CLWR1', 'NSR4', 'BPTR', 'NR', 'BRZR', 'RSC', 'MDC', 'TC', 'STRW', 'WDC', 'CNJC', 'NSR5', 'WMC', 'STUR', 'RDWR', 'NSR6', 'AC', 'VR', 'OMR1', 'PC1', 'PC2', 'BRC', 'WK', 'WNR', 'BYR1', 'BYR2', 'SMR', 'OMR2', 'MQC', 'LBR1', 'LBR2', 'LBR3', 'LBR4', 'OMR3', 'EC', 'WR1', 'WR2', 'SYR', 'PR1', 'PR2', 'PR3', 'FTC', 'RDR1', 'BBC', 'JR', 'RR', 'LRDR', 'MR', 'RDR2', 'WOC', 'BMR', 'RDR3', 'RDR4', 'THC1', 'THC2', 'KC1', 'KC2', 'MC1', 'MC2', 'RBR1', 'RBR2', 'MTC', 'BRYC', 'RDR5', 'SSR-1', 'SPC', 'RK', 'SSR-2']


# Sandbox

## 1. Practicing grouping

In [55]:
df = data.groupby(['Station', 'VariableName']).agg({
    'MeasurementValue': ['mean', 'std', 'count'],
    'SampleDateTime': ['min', 'max'],
    'Censored': ['sum']
}).unstack(['Station', 'VariableName']).reset_index()

/var/folders/_g/mxhqq21x7359p_ly0jzzd66m0000gn/T/ipykernel_12856/3913926887.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = data.groupby(['Station', 'VariableName']).agg({


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869736 entries, 0 to 869735
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   level_0       869736 non-null  object  
 1   level_1       869736 non-null  object  
 2   Station       869736 non-null  category
 3   VariableName  869736 non-null  object  
 4   0             437673 non-null  object  
dtypes: category(1), object(4)
memory usage: 27.4+ MB


In [ ]:
# Checking a random value
data[(data['Station'] == 'Sunwapta River, At Wsc Gauge 0.1 Km D/S Of Sunwapta Lake') &
                (data['VariableName'] == 'Boron Total Recoverable')]['MeasurementValue'].mean()

np.float64(1.1885714285714286)